In [37]:
#!/usr/bin/env python
# coding: utf-8

# This code plots the parameter trace during a parameter estimation process.
# Note: This code plots the incomplete trace of samples because it reads sampels from ostOutput.txt.
# Only the parameter sets that improve the objective function in comparison with the previous parameter set are plotted.

# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt

import os, sys, argparse, datetime, shutil
from glob import glob
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt 
import xarray as xr
import pandas as pd
from matplotlib.dates import DateFormatter
from tqdm import tqdm

# Function to extract a given setting from the configuration file
def read_from_control(control_file, setting):
    
    # Open 'control_active.txt' and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('|',1)[1].split('#',1)[0].strip() 
    # Return this value    
    return substring

# Function to extract a given setting from the summa and mizuRoute manager/control files
def read_from_summa_route_control(control_file, setting):

    # Open fileManager.txt or route_control and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('!',1)[0].strip().split(None,1)[1].strip("'")
    # Return this value    
    return substring

def is_number(s):
    try:
        float(s)
        return True 
    except (ValueError,AttributeError):
        return False 
                    
def read_param_sample_from_ostModel(ostModel):    
    sample_ids=[]
    objs = []
    sample_values = []
    
    data = np.loadtxt(ostModel, skiprows=1)
    sample_ids = data[:,0]
    objs = data[:,1]
    sample_values = data[:,2:]
    return  sample_ids,objs,sample_values

# Function to extract the param default values and bounds from basinParam and localParam.txt.
def read_basinParam_localParam(filename):
    param_names = []
    param_default = []
    param_min = []
    param_max =[]
    with open (filename, 'r') as f:
        for line in f:
            line=line.strip()
            if line and not line.startswith('!') and not line.startswith("'"):
                splits=line.split('|')
                if isinstance(splits[0].strip(), str):
                    param_names.append(splits[0].strip())
                    param_default.append(str_to_float(splits[1].strip()))
                    param_min.append(str_to_float(splits[2].strip()))
                    param_max.append(str_to_float(splits[3].strip()))
    return param_names, param_default, param_min, param_max

# Function to convert data from Fortran format to scientific format.
def str_to_float(data_str):
    if 'd' in data_str:
        x = data_str.split('d')[0]+'e'+data_str.split('d')[1]
        return float(x)
    else:
        return float(data_str)

# main
if __name__ == '__main__':
    
    # ----------------------------- Settings ------------------------------        
    # calib inputs
    root_path = '/home/h294liu/project/proj/5_summaCalib'  # root path where parameter estimation will be stored.
    domain_name = 'BowAtBanff' #'BowAtBanff_LA_calib' #'BowAtBanff'

    calib_basename = 'DDS' #SCE #GA #DDS #GLUE  
    default_model_folder = 'BowAtBanff_default' #owAtBanff_LA' #'BowAtBanff_default'
    outFilePrefix = 'run1'

    # input paths
    calib_output_path = os.path.join(root_path, domain_name+'_'+calib_basename)
    default_output_path = os.path.join(root_path, default_model_folder)
    direct_param_list = []
    
    param_txt = os.path.join(calib_output_path, 'analysis', '8_plot_param_trace_collect',
                             '%s_param_samples.txt'%(calib_basename)) # all param information
    
    # identify plot output path and file
    output_path = os.path.join(calib_output_path, 'analysis', '9_plot_obj_param_sensitivity')
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    ofile_fig = os.path.join(output_path, '%s_obj_param.png'%(calib_basename))   # output plot figure
   
    # --------------------------- End settings -----------------------------        
    
#     # 1. Read deafault model information (control file, param, obj)    
#     control_file = os.path.join(default_output_path, 'calib','control_active.txt')
#     domain_name = read_from_control(control_file, 'domain_name')
#     domain_path = os.path.join(root_path, domain_name)

#     # hydrologic model path, settings, fileManager.txt, trialParam.nc.
#     model_dst_path = read_from_control(control_file, 'model_dst_path')
#     if model_dst_path == 'default':
#         model_dst_path = os.path.join(domain_path, 'model')

#     summa_settings_relpath = read_from_control(control_file, 'summa_settings_relpath')
#     summa_settings_path = os.path.join(model_dst_path, summa_settings_relpath)

#     summa_filemanager = read_from_control(control_file, 'summa_filemanager')
#     summa_filemanager = os.path.join(summa_settings_path, summa_filemanager)
    
#     trialParamFile = read_from_summa_route_control(summa_filemanager, 'trialParamFile')
#     trialParamFile_priori = trialParamFile.split('.nc')[0] + '.priori.nc' # a priori param file
#     trialParamFile_priori = os.path.join(summa_settings_path, trialParamFile_priori)
    
#     # 2. Read parameter names and ranges from default model
#     print('Read parameter names and ranges.')
#     # (1) read parameter names from summa_parameter_bounds.txt
#     # param_names is a list of params corresponding to multiplier, used to update trialParam file.
#     param_names = list(np.loadtxt(os.path.join(default_output_path, 'calib', 'summa_parameter_bounds.txt'), 
#                                   usecols=[0], dtype='str', delimiter=','))
#     param_range = np.loadtxt(os.path.join(default_output_path, 'calib', 'summa_parameter_bounds.txt'), 
#                              usecols=[2,3], delimiter=',')
    
#     # (2) read parameter ranges from localParam.txt and basinParam.txt.
#     basinParam = read_from_summa_route_control(summa_filemanager, 'globalGruParamFile')
#     localParam = read_from_summa_route_control(summa_filemanager, 'globalHruParamFile')

#     basinParam = os.path.join(summa_settings_path, basinParam)
#     localParam = os.path.join(summa_settings_path, localParam)

#     basin_param_names, basin_param_default, basin_param_min, basin_param_max = read_basinParam_localParam(basinParam)    
#     local_param_names, local_param_default, local_param_min, local_param_max = read_basinParam_localParam(localParam)
 
#     # 3. Gather a priori param values and model performance for plot
#     priori_param_values = np.zeros((Nparam,)) #store the a priori param values in array
#     with nc.Dataset(trialParamFile_priori, 'r') as src:
#         for iParam in range(Nparam):
#             param_name = output_param_names[iParam]
#             if param_name in src.variables.keys():  
#                 param_ma_priori    = src.variables[param_name][:]     # priori param value mask array 
#                 param_value_priori = param_ma_priori.data.flat[0]     # value of the first HRU/GRU 
#                 priori_param_values[iParam] = param_value_priori
#             else:
#                 priori_param_values[iParam] = np.nan
                
#     # 4. Read parameter samples
#     with open(param_txt) as f:
#         firstLine = f.readline()
#         output_param_names = firstLine.strip().split('\t')[2:]    
#     data = np.loadtxt(param_txt, skiprows=1).astype('float64')
    
#     sample_ids_concat = data[:,0]
#     objs_concat = data[:,1]
#     output_param_values = data[:,2:]

#     (Nsample,Nparam) = np.shape(output_param_values) 
    
#     # best param set index
#     best_obj = np.min(objs_concat)
#     best_indices = np.argwhere(objs_concat == best_obj)
    
#     # top 10% param set index
#     obj_top_threshold = np.percentile(objs_concat, 10) 
#     top_indices = np.argwhere(objs_concat <= obj_top_threshold)
    
    # 5. Plot objective function and parameter trace.
    print('Plot parameter traces.')
    col_num = 4        
    row_num = int(np.ceil(Nparam/float(col_num)))
    
    fig, ax = plt.subplots(row_num,col_num, figsize=(3.0*col_num, 3.0*0.75*row_num), constrained_layout=True)
    fig.suptitle('%s'%(calib_basename), fontsize='large', fontweight='bold')
    
    dpi_value=80
    ms_sample = 1 # marker size for samples
    ms_highlight = 5 # marker size for highlight points   
    
    for i in range(row_num):
        for j in range(col_num):
            
            subplot_count = i*col_num + j
            param_index = subplot_count - 1
            
            if subplot_count <= Nparam: 
                param_name = output_param_names[param_index]
                
                # plot all parameters
                ax[i,j].plot(output_param_values[:, param_index], objs_concat, color='b',marker='o',
                             linewidth=0.0, markersize=ms_sample, alpha=0.7)
                
                # plot top 10% parameters
                top_param_values = output_param_values[top_indices, param_index]
                ax[i,j].plot(top_param_values, objs_concat[top_indices], 
                             'o', color="r", markersize=ms_sample, alpha=0.7);                

                # plot the a priori and best samples 
                ax[i,j].plot(priori_param_values[param_index], objs_concat[0], 
                             'D', markerfacecolor="none", markeredgecolor="k", markersize=ms_highlight, markeredgewidth=1.5);

                best_param_values = output_param_values[best_indices, param_index]
                ax[i,j].plot(best_param_values, objs_concat[best_indices], 
                             's', markerfacecolor="none", markeredgecolor="g", markersize=ms_highlight, markeredgewidth=1.5);                
                                    
                # ylimit, axis, label, title
                if param_name in param_names:
                    param_min = param_range[param_names.index(param_name)][0]
                    param_max = param_range[param_names.index(param_name)][1]
                elif param_name in local_param_names:
                    param_min = local_param_min[local_param_names.index(param_name)]
                    param_max = local_param_max[local_param_names.index(param_name)]
                elif param_name in basin_param_names:
                    param_min = basin_param_min[basin_param_names.index(param_name)]
                    param_max = basin_param_max[basin_param_names.index(param_name)]            
                ax[i,j].set_xlim([param_min, param_max])

                title_str = '('+chr(ord('a') + subplot_count) +') ' + param_name
                ax[i,j].set_title(title_str, fontsize='medium', fontweight='semibold')
                if j == 0:
                    ax[i,j].set_ylabel('Obj. function (-KGE)', fontsize='medium')
                ax[i,j].set_xlabel('Parameter value', fontsize='medium')
                            
            # blank subplots           
            else: 
                # plot legend
                if subplot_count == Nparam+1: 
                    ax[i,j].set_frame_on(False)
                    ax[i,j].get_xaxis().set_visible(False)
                    ax[i,j].get_yaxis().set_visible(False)
                    ax[i,j].plot(np.nan, np.nan, 'o', markerfacecolor="blue", markeredgecolor='none', label = 'Parameter sample')
                    ax[i,j].plot(np.nan, np.nan, 'o', markerfacecolor="r", markeredgecolor='none', label = 'Top 10% parameter sample')
                    ax[i,j].plot(np.nan, np.nan, 'D', markerfacecolor="none", markeredgecolor="k", markersize=ms_highlight, 
                                 markeredgewidth=1.5, label='A priori value') #darkorange
                    ax[i,j].plot(np.nan, np.nan, 's', markerfacecolor="none", markeredgecolor="g", markersize=ms_highlight, 
                                 markeredgewidth=1.5, label='Best value')
                    ax[i,j].legend(loc = 'center left')
                else:
                    ax[i,j].axis('off')

    plt.rc('xtick',labelsize='medium')
    plt.rc('ytick',labelsize='medium')                
    fig.savefig(ofile_fig, dpi=dpi_value)
    plt.close(fig)  


Plot parameter traces.
